In [1]:
import serial
import time
import struct
import threading
import time 

# Serial NTP Server Class

In [ ]:
class serialNtpServer():
    '''A class that implements an NTP Server with serial interface. This has been tested with SiK serial radio modules'''
    
    def __init__(self, port = '/dev/ttyUSB1', baudRate = 57600):
        '''
        The constructor of the Serial NTP Server class. With this, the NTP server starts
        @param: port:
        The serial ID of the port we want to use for the server (The port representing the radio module
        @param: baudRate:
        The communication boadrate of the serial interface
        @param: record:
        '''
        self.port = serial.Serial(port ,baudRate, timeout = 2)        
        self.running = True
        self.receiveTread = threading.Thread(target=self.receivingThread, args=())
        self.receiveTread.start()
        
    def receivingThread(self):
        while self.running:
            data = self.port.read(8)
            if len(data) == 8:
                #record the time of the recpetion of the request
                self.stamp2 = time.time_ns();
                self.stamp1 = int(struct.unpack('Q', data)[0])
                self.transmitResponse()
                
    def transmitResponse(self):
        #Send back the response containing the time of reception of the requiest and time we send the response
        payload = struct.pack('3Q', self.stamp1, self.stamp2, time.time_ns())
        self.port.write(payload)
                
    def serverStop(self):
        self.running = False
        self.receiveTread.join()
        self.port.close()
        

## Start the Server

In [ ]:
server = serialNtpServer()